In [96]:
import torch
from transformers import RobertaTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, TensorDataset, random_split
from tqdm import tqdm

In [97]:
import pandas as pd
from transformers import RobertaTokenizer

# Load the CSV file
df = pd.read_csv('C:\\Users\\Owner\\Desktop\\Inefficient_Rendering\\Inefficient_Rendering.csv')

# Specify the model name or identifier
model_name = 'microsoft/graphcodebert-base'

# Load the correct tokenizer (RobertaTokenizer)
tokenizer = RobertaTokenizer.from_pretrained(model_name)

# Tokenize the code snippets and convert them to tensors
inputs = tokenizer(
    df['FPath'].tolist(),
    padding=True,
    truncation=True,
    return_tensors='pt'
)

# Encode labels
labels = df['Smell'].apply(lambda x: 1 if x == "Inefficient Rendering" else 0)
print(labels)


0      1
1      1
2      1
3      1
4      1
      ..
445    0
446    0
447    0
448    0
449    0
Name: Smell, Length: 450, dtype: int64


In [98]:
from sklearn.model_selection import train_test_split

# Split your dataset into training (70%) and temporary (30%) sets
train_temp, test = train_test_split(df, test_size=0.30, random_state=42)

# Split the temporary set into validation (15%) and final test (15%) sets
validation, test = train_test_split(test, test_size=0.50, random_state=42)

# Print the number of samples in each set
print(f"Train set size: {len(train_temp)}")
print(f"Validation set size: {len(validation)}")
print(f"Test set size: {len(test)}")


Train set size: 315
Validation set size: 67
Test set size: 68


In [99]:
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, TensorDataset
from transformers import RobertaForSequenceClassification, AdamW, get_linear_schedule_with_warmup
import torch.nn as nn
import numpy as np

# Convert data to PyTorch tensors
train_inputs = torch.tensor(inputs['input_ids'][train_temp.index])
train_labels = torch.tensor(np.array(labels[train_temp.index]))  # Convert to NumPy array first

validation_inputs = torch.tensor(inputs['input_ids'][validation.index])
validation_labels = torch.tensor(np.array(labels[validation.index]))

test_inputs = torch.tensor(inputs['input_ids'][test.index])
test_labels = torch.tensor(np.array(labels[test.index]))


# Create DataLoader for training, validation, and test sets
batch_size = 32  # Adjust as needed
train_dataset = TensorDataset(train_inputs, train_labels)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

validation_dataset = TensorDataset(validation_inputs, validation_labels)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size)

test_dataset = TensorDataset(test_inputs, test_labels)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Initialize the model
model = RobertaForSequenceClassification.from_pretrained('microsoft/graphcodebert-base', num_labels=2)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=1e-5)
num_epochs = 5

# Learning rate scheduler
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=len(train_loader) * num_epochs
)
    
# Initialize lists to store training predictions and true labels
training_predictions = []
training_true_labels = []

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    for batch in train_loader:
        inputs, labels = batch
        optimizer.zero_grad()

        outputs = model(inputs, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        scheduler.step()

        # Calculate training predictions and true labels for accuracy calculation
        train_logits = outputs.logits
        train_probabilities = torch.softmax(train_logits, dim=1)
        train_predicted_labels = torch.argmax(train_probabilities, dim=1)

        training_predictions.extend(train_predicted_labels.cpu().numpy())
        training_true_labels.extend(labels.cpu().numpy())

    average_loss = total_loss / len(train_loader)

    # Compute training accuracy
    training_accuracy = accuracy_score(training_true_labels, training_predictions)


    # Validation
    model.eval()  # Set the model to evaluation mode
    validation_loss = 0.0
    validation_predictions = []
    validation_true_labels = []

    with torch.no_grad():
        for val_batch in validation_loader:
            val_inputs, val_labels = val_batch
            val_outputs = model(val_inputs)
            val_logits = val_outputs.logits

            # Calculate the loss based on the model's outputs and labels
            val_loss = criterion(val_logits, val_labels).item()
            validation_loss += val_loss

            val_probabilities = torch.softmax(val_logits, dim=1)
            val_predicted_labels = torch.argmax(val_probabilities, dim=1)

            validation_predictions.extend(val_predicted_labels.cpu().numpy())
            validation_true_labels.extend(val_labels.cpu().numpy())

        # Compute validation accuracy
        validation_accuracy = accuracy_score(validation_true_labels, validation_predictions)
        validation_loss /= len(validation_loader)



    # Test
    test_loss = 0.0
    test_predictions = []
    test_true_labels = []

    with torch.no_grad():
        for test_batch in test_loader:
            test_inputs, test_labels = test_batch
            test_outputs = model(test_inputs)
            test_logits = test_outputs.logits

            # Calculate the loss based on the model's outputs and labels
            test_loss += criterion(test_logits, test_labels).item()

            test_probabilities = torch.softmax(test_logits, dim=1)
            test_predicted_labels = torch.argmax(test_probabilities, dim=1)

            test_predictions.extend(test_predicted_labels.cpu().numpy())
            test_true_labels.extend(test_labels.cpu().numpy())

        test_loss /= len(test_loader)
        test_accuracy = accuracy_score(test_true_labels, test_predictions)

    # Print epoch results
    print(f"Epoch {epoch + 1}/{num_epochs}")
    print(f"Train Loss: {average_loss:.4f} | Train Accuracy: {training_accuracy:.4f}")
    print(f"Validation Loss: {validation_loss:.4f} | Validation Accuracy: {validation_accuracy:.4f}")
    print(f"Test Loss: {test_loss:.4f} | Test Accuracy: {test_accuracy:.4f}")


C:\Users\Owner\AppData\Local\Temp/ipykernel_12360/2540781221.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_inputs = torch.tensor(inputs['input_ids'][train_temp.index])
C:\Users\Owner\AppData\Local\Temp/ipykernel_12360/2540781221.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  validation_inputs = torch.tensor(inputs['input_ids'][validation.index])
C:\Users\Owner\AppData\Local\Temp/ipykernel_12360/2540781221.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_inputs = torch.tensor(inputs['input_ids'][test.index])
Some

Epoch 1/5
Train Loss: 0.4979 | Train Accuracy: 0.8730
Validation Loss: 0.3228 | Validation Accuracy: 0.9403
Test Loss: 0.2713 | Test Accuracy: 0.9706
Epoch 2/5
Train Loss: 0.3007 | Train Accuracy: 0.9111
Validation Loss: 0.2160 | Validation Accuracy: 0.9403
Test Loss: 0.1491 | Test Accuracy: 0.9706
Epoch 3/5
Train Loss: 0.2367 | Train Accuracy: 0.9238
Validation Loss: 0.1862 | Validation Accuracy: 0.9403
Test Loss: 0.1137 | Test Accuracy: 0.9706
Epoch 4/5
Train Loss: 0.2181 | Train Accuracy: 0.9302
Validation Loss: 0.1755 | Validation Accuracy: 0.9403
Test Loss: 0.1056 | Test Accuracy: 0.9706
Epoch 5/5
Train Loss: 0.2155 | Train Accuracy: 0.9340
Validation Loss: 0.1729 | Validation Accuracy: 0.9403
Test Loss: 0.1037 | Test Accuracy: 0.9706


In [ ]:
# import torch
# from torch.utils.data import DataLoader, TensorDataset
# from transformers import RobertaForSequenceClassification, AdamW, get_linear_schedule_with_warmup
# import torch.nn as nn
# import numpy as np

# # Convert data to PyTorch tensors
# train_inputs = torch.tensor(inputs['input_ids'][train_temp.index])
# train_labels = torch.tensor(np.array(labels[train_temp.index]))  # Convert to NumPy array first

# validation_inputs = torch.tensor(inputs['input_ids'][validation.index])
# validation_labels = torch.tensor(np.array(labels[validation.index]))

# test_inputs = torch.tensor(inputs['input_ids'][test.index])
# test_labels = torch.tensor(np.array(labels[test.index]))


# # Create DataLoader for training, validation, and test sets
# batch_size = 32  # Adjust as needed
# train_dataset = TensorDataset(train_inputs, train_labels)
# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# validation_dataset = TensorDataset(validation_inputs, validation_labels)
# validation_loader = DataLoader(validation_dataset, batch_size=batch_size)

# test_dataset = TensorDataset(test_inputs, test_labels)
# test_loader = DataLoader(test_dataset, batch_size=batch_size)

# # Initialize the model
# model = RobertaForSequenceClassification.from_pretrained('microsoft/graphcodebert-base', num_labels=2)

# # Define loss function and optimizer
# criterion = nn.CrossEntropyLoss()
# optimizer = AdamW(model.parameters(), lr=1e-5)
# num_epochs = 5

# # Learning rate scheduler
# scheduler = get_linear_schedule_with_warmup(
#     optimizer,
#     num_warmup_steps=0,
#     num_training_steps=len(train_loader) * num_epochs
# )

# for epoch in range(num_epochs):
#     model.train()
#     total_loss = 0.0

#     for batch in train_loader:
#         inputs, labels = batch
#         optimizer.zero_grad()

#         outputs = model(inputs, labels=labels)
#         loss = outputs.loss
#         total_loss += loss.item()

#         loss.backward()
#         optimizer.step()
#         scheduler.step()

#     average_loss = total_loss / len(train_loader)
#     print(f"Epoch {epoch+1}/{num_epochs} - Average Loss: {average_loss:.4f}")

# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# # Initialize lists to store predictions and true labels
# validation_predictions = []
# validation_true_labels = []

# test_predictions = []
# test_true_labels = []

# # Evaluation loop for the validation set
# with torch.no_grad():
#     for batch in validation_loader:
#         inputs, labels = batch
#         outputs = model(inputs)
#         logits = outputs.logits
#         probabilities = torch.softmax(logits, dim=1)
#         predicted_labels = torch.argmax(probabilities, dim=1)
        
#         validation_predictions.extend(predicted_labels.cpu().numpy())
#         validation_true_labels.extend(labels.cpu().numpy())

# # Evaluation loop for the test set
# with torch.no_grad():
#     for batch in test_loader:
#         inputs, labels = batch
#         outputs = model(inputs)
#         logits = outputs.logits
#         probabilities = torch.softmax(logits, dim=1)
#         predicted_labels = torch.argmax(probabilities, dim=1)
        
#         test_predictions.extend(predicted_labels.cpu().numpy())
#         test_true_labels.extend(labels.cpu().numpy())

# # Calculate evaluation metrics for validation set
# accuracy_val = accuracy_score(validation_true_labels, validation_predictions)
# precision_val = precision_score(validation_true_labels, validation_predictions)
# recall_val = recall_score(validation_true_labels, validation_predictions)
# f1_val = f1_score(validation_true_labels, validation_predictions)

# # Calculate evaluation metrics for test set
# accuracy_test = accuracy_score(test_true_labels, test_predictions)
# precision_test = precision_score(test_true_labels, test_predictions)
# recall_test = recall_score(test_true_labels, test_predictions)
# f1_test = f1_score(test_true_labels, test_predictions)

# # Print the evaluation results
# print("Validation Set Metrics:")
# print(f"Accuracy: {accuracy_val:.4f}")
# print(f"Precision: {precision_val:.4f}")
# print(f"Recall: {recall_val:.4f}")
# print(f"F1-Score: {f1_val:.4f}")

# print("\nTest Set Metrics:")
# print(f"Accuracy: {accuracy_test:.4f}")
# print(f"Precision: {precision_test:.4f}")
# print(f"Recall: {recall_test:.4f}")
# print(f"F1-Score: {f1_test:.4f}")
    